<a href="https://colab.research.google.com/github/aguilargallardo/Unit_2_Build/blob/master/Rap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://raw.githubusercontent.com/aguilargallardo/DS-Unit-2-Applied-Modeling/master/data/SpotifyFeatures.csv"

--2020-01-08 18:55:28--  https://raw.githubusercontent.com/aguilargallardo/DS-Unit-2-Applied-Modeling/master/data/SpotifyFeatures.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33709086 (32M) [text/plain]
Saving to: ‘SpotifyFeatures.csv.5’

SpotifyFeatures.csv 100%[===================>]  32.15M   165MB/s    in 0.2s    

2020-01-08 18:55:29 (165 MB/s) - ‘SpotifyFeatures.csv.5’ saved [33709086/33709086]



In [0]:
import pandas as pd
df = pd.read_csv('SpotifyFeatures.csv')

In [3]:
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [4]:
df.genre.value_counts()

Comedy              9681
Soundtrack          9646
Indie               9543
Jazz                9441
Pop                 9386
Electronic          9377
Children’s Music    9353
Folk                9299
Hip-Hop             9295
Rock                9272
Alternative         9263
Classical           9256
Rap                 9232
World               9096
Soul                9089
Blues               9023
R&B                 8992
Anime               8936
Reggaeton           8927
Ska                 8874
Reggae              8771
Dance               8701
Country             8664
Opera               8280
Movie               7806
Children's Music    5403
A Capella            119
Name: genre, dtype: int64

In [5]:
df.genre.nunique()

27

In [0]:
# df['id'] = df.groupby(['artist_name']).ngroup()

# RAP

In [0]:
Rap = df[df.genre == 'Rap']

In [8]:
Rap.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
86951,Rap,Post Malone,Wow.,6MWtB6iiXyIwun0YzU6DFP,99,0.16300,0.833,149520,0.539,0.000002,B,0.1010,-7.399,Minor,0.1780,99.947,4/4,0.385
86952,Rap,J. Cole,MIDDLE CHILD,2JvzF1RMd7lE3KmFlsyZD8,96,0.14900,0.837,213594,0.364,0.000000,G#,0.2710,-11.713,Major,0.2760,123.984,4/4,0.463
86953,Rap,Post Malone,Sunflower - Spider-Man: Into the Spider-Verse,3KkXRkHbMCARz0aVfEt68P,97,0.55600,0.760,158040,0.479,0.000000,D,0.0703,-5.574,Major,0.0466,89.911,4/4,0.913
86954,Rap,Travis Scott,SICKO MODE,2xLMifQCjDGFmkHkpNLD9h,94,0.00513,0.834,312820,0.730,0.000000,G#,0.1240,-3.714,Major,0.2220,155.008,4/4,0.446
86955,Rap,Meek Mill,Going Bad (feat. Drake),2IRZnDFmlqMuOrYOLnZZyc,95,0.25900,0.889,180522,0.496,0.000000,E,0.2520,-6.365,Minor,0.0905,86.003,4/4,0.544


In [0]:
Rap = Rap.sort_values(by='popularity', ascending=False)

In [10]:
Rap.head(20), Rap.shape

(      genre             artist_name  ... time_signature valence
 86951   Rap             Post Malone  ...            4/4   0.385
 86953   Rap             Post Malone  ...            4/4   0.913
 86952   Rap                 J. Cole  ...            4/4   0.463
 86955   Rap               Meek Mill  ...            4/4   0.544
 86954   Rap            Travis Scott  ...            4/4   0.446
 86965   Rap             Post Malone  ...            4/4   0.374
 86961   Rap             Kodak Black  ...            4/4   0.504
 86957   Rap               21 Savage  ...            4/4   0.274
 86956   Rap                Blueface  ...            4/4   0.391
 86958   Rap                Lil Baby  ...            4/4   0.358
 86963   Rap              Juice WRLD  ...            4/4   0.218
 86967   Rap               Sheck Wes  ...            4/4   0.261
 86970   Rap            XXXTENTACION  ...            4/4   0.473
 86960   Rap              Juice WRLD  ...            4/4   0.535
 86962   Rap  A Boogie Wi

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(Rap, train_size=.8)

In [0]:
target = 'popularity'
feature = ['acousticness','danceability','duration_ms',
           'energy','instrumentalness','liveness','loudness',
           'speechiness','tempo','valence']

In [0]:
X_train = train[feature]
X_test = test[feature]

y_test = test[target]
y_train = train[target]

In [0]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler(),
    RandomForestRegressor(),
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

In [15]:
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestregressor__n_estimators': randint(50, 500), 
    'randomforestregressor__max_depth': [5, 10, 15, 20, None], 
    'randomforestregressor__max_features': uniform(0, 1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   41.6s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   49.7s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.6min finished


In [0]:
y_pred = pipeline.predict(X_test)

In [17]:
print('Best hyperparameters', search.best_params_)
print('MAE', -search.best_score_)

Best hyperparameters {'randomforestregressor__max_depth': 15, 'randomforestregressor__max_features': 0.048671349830147426, 'randomforestregressor__n_estimators': 393, 'simpleimputer__strategy': 'mean'}
MAE 6.374655946251711


In [18]:
y_pred.shape

(1847,)

In [19]:
X_test.shape

(1847, 10)

In [0]:
# Predictions = pd.DataFrame({'Predict' : y_pred, 'id' : X_test['id'] })

In [21]:
X_test

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
120075,0.04630,0.817,86507,0.450,0.000000,0.0774,-5.557,0.2100,90.104,0.7690
117455,0.11500,0.551,282173,0.747,0.000000,0.0895,-7.911,0.2540,88.937,0.2050
120091,0.16600,0.793,207256,0.589,0.000000,0.1230,-5.268,0.1910,149.915,0.4980
115297,0.00022,0.524,184867,0.939,0.000000,0.6230,-3.994,0.1130,150.184,0.6610
121674,0.72700,0.837,226052,0.642,0.000000,0.0999,-8.924,0.3460,94.986,0.5680
...,...,...,...,...,...,...,...,...,...,...
116448,0.04710,0.717,262137,0.693,0.000000,0.1310,-8.315,0.0339,130.000,0.4970
115681,0.00945,0.818,190222,0.573,0.000005,0.1140,-8.299,0.2200,134.982,0.3040
121671,0.08520,0.727,151933,0.645,0.000012,0.4050,-5.994,0.0530,119.957,0.2810
119342,0.42000,0.589,95709,0.484,0.038500,0.1090,-11.613,0.0742,139.998,0.0726


##Feature Importance

In [0]:

transformers = make_pipeline( 
    SimpleImputer(strategy='median')
)

In [23]:
X_train_transformed = transformers.fit_transform(X_train)
X_test_transformed = transformers.transform(X_test)

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_transformed, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [24]:
!pip install eli5

In [25]:
import eli5
from eli5.sklearn import PermutationImportance

# 1. Calculate permutation importances
permuter = PermutationImportance(
    search,
    scoring="neg_mean_absolute_error",
    n_iter=5,
    random_state=42
)

permuter.fit(X_test_transformed, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


PermutationImportance(cv='prefit',
                      estimator=RandomizedSearchCV(cv=3, error_score=nan,
                                                   estimator=Pipeline(memory=None,
                                                                      steps=[('simpleimputer',
                                                                              SimpleImputer(add_indicator=False,
                                                                                            copy=True,
                                                                                            fill_value=None,
                                                                                            missing_values=nan,
                                                                                            strategy='median',
                                                                                            verbose=0)),
                                                            

In [26]:
feature_names = X_test.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()

acousticness        0.002348
liveness            0.004553
tempo               0.008726
valence             0.014845
instrumentalness    0.016273
duration_ms         0.018550
danceability        0.023589
energy              0.042050
speechiness         0.046724
loudness            0.062386
dtype: float64

In [27]:
# 2. Display permutation importances
eli5.show_weights(
    permuter, 
    top=None, # show permutation importances for all features
    feature_names=feature_names # must be a list
)

Weight,Feature
0.0624 ± 0.0247,loudness
0.0467 ± 0.0180,speechiness
0.0420 ± 0.0283,energy
0.0236 ± 0.0182,danceability
0.0186 ± 0.0136,duration_ms
0.0163 ± 0.0068,instrumentalness
0.0148 ± 0.0043,valence
0.0087 ± 0.0222,tempo
0.0046 ± 0.0190,liveness
0.0023 ± 0.0083,acousticness


In [28]:
print('Shape before removing features:', X_train.shape)

Shape before removing features: (7385, 10)


In [0]:
minimum_importance = 0
mask = permuter.feature_importances_ > minimum_importance
features = X_train.columns[mask]
X_train = X_train[features]

In [30]:
features

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence'],
      dtype='object')

In [31]:
print('Shape after removing features:', X_train.shape)

Shape after removing features: (7385, 10)


In [32]:
X_test = X_test[features]

pipeline = make_pipeline( 
    SimpleImputer(strategy='median'), 
    RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
)

# Fit on train, score on val
pipeline.fit(X_train, y_train)
print('R2 Scored: ', pipeline.score(X_test, y_test))

R2 Scored:  0.009771844125555429


In [33]:
param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestregressor__n_estimators': randint(50, 500), 
    'randomforestregressor__max_depth': [5, 10, 15, 20, None], 
    'randomforestregressor__max_features': uniform(0, 1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.9min finished


In [34]:
y_pred = pipeline.predict(X_test)
print('Best hyperparameters', search.best_params_)
print('MAE', -search.best_score_)

Best hyperparameters {'randomforestregressor__max_depth': 5, 'randomforestregressor__max_features': 0.6671858201035501, 'randomforestregressor__n_estimators': 309, 'simpleimputer__strategy': 'mean'}
MAE 6.378107034623702
